# Germany Rental Prediction - Cleaning, EDA and Prediction

## Purpose from this kernel.

I've travel from SEA and I don't know how much apartment in the Berlin should cost and it's really tough to find an apartment while I'm staying in Germany for my Master Degree. Furthermore, I need something for my Data Science Portfolio for the job application after graduation. So why not build something from the scratch with the dataset on the Kaggle

So this kernel will be well written than my previous kernel for other people and using what I've learnt in my master course and other online resources to produce something that will be practical for the real environment.

## What we expected from this kernel.
- Data cleaning to clear the outliers and remove columns that doesn't have high correlation to the prediction
- Create virtualization to have a better understanding of the data of the rental in Germany.
- Feature engineering from the original variable to create a better model
- Create a tool that estimate the house cost predicted by many variables

# Basic data handling and inspection

Import all important libraries in this kernel

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import time
import datetime
from datetime import date
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import lightgbm as lgb
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_columns', None)

Load the dataset to the kernel

In [ ]:
df = pd.read_csv('../input/apartment-rental-offers-in-germany/immo_data.csv')

### Simple Inspection

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print(f'Number of columns is:',df.shape[1],'and rows is:',df.shape[0])

# Cleaning Job

The boring part of this kernel. It might a bit confuse but what we're trying in this section is dealing with many things such as
- Outliers
- Missing Data
- Drop columns
- Etc.
The result of this part is we will have a better dataset to analyze, virtualization and making a prediction.

## Dealing with the missing values

When we're working on any datasets, we need to check on the missing values to make sure the data is ready or not for further analyzation and virutalization.

Create the function to show the top 20 missing values by include number of missing values and percentage of it.

In [ ]:
def missing_values(df,norows):   # input by the df and the number of rows that you want to show
    total = df.isnull().sum().sort_values(ascending=False)
    percent = ((df.isnull().sum().sort_values(ascending=False)/df.shape[0])*100).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return(missing_data.head(norows))


In [ ]:
missing_values(df,20) # we use the df and the number of rows to show is 20

As we can see it contains a lot of missing value in some columns so I decide to remove all of the columns that contain missing value more than 20%

In [ ]:
missing_data = missing_values(df,20)
# drop the data where the columns contains more than 30%
df = df.drop((missing_data[missing_data['Percent'] > 30]).index,1)

Because I want to predict rental price ('totalRent') so I should drop all the rows that doesn't consist totalRent

In [ ]:
df.dropna(subset=['totalRent'],inplace=True)

### Delete columns that doesn't contain useful information

In [ ]:
df.head()

### Drop and delete what we wouldn't use

In [ ]:
df.drop(columns=['livingSpaceRange','street','description','facilities','geo_krs','geo_plz','scoutId','regio1','telekomUploadSpeed','telekomTvOffer','pricetrend','regio3','noRoomsRange','picturecount','geo_bln','date',\
    'houseNumber','streetPlain','firingTypes','yearConstructedRange'],inplace=True)

Let's check the missing data in this dataframe again before making any other decision.

In [ ]:
missing_values(df,10)

### Other condition
Take a deeper analysis in the condition variable and I fill all of the missing value into 'Other'

In [ ]:
df['condition'].fillna("Other", inplace=True) # fill the NA by Other
df['condition'].value_counts()

The last 3 is not the good condition for the apartnebt fubder so I will group it in 'Other'

In [ ]:
otherscondition = df['condition'].value_counts().tail(3).index

othersregion = list(df['condition'].value_counts().tail(3).index)
def editcondition(dflist):
    if dflist in otherscondition:
        return 'Other'
    else:
        return dflist

df['condition'] =df['condition'].apply(editcondition)
df['condition'].value_counts()

Fill NA of 'yearConstructed' with the mean of each type of condition 'condition' because from my perspective if the apartment is not fully_renovated or refurbished it means that it should have a lot of usage year.

In [ ]:
df["yearConstructed"] = df['yearConstructed'].fillna(df.groupby('condition')['yearConstructed'].transform('mean')).round(0)

Create the new variables to tell the duration since last renovated or built til the today

In [ ]:
df['numberOfYear'] = date.today().year - df["yearConstructed"]


### Other region
In other region, It might not have sufficient data so I group them together and put it in Other variable.

In [ ]:
othersregion = list(df['regio2'].value_counts().iloc[20:,].index)
def edit_region(dflist):
    if dflist in othersregion:
        return 'Other'
    else:
        return dflist

df['regio2'] =df['regio2'].apply(edit_region)
df['regio2'].value_counts()

### Selecting only highest 20 city by quantity of data
I choose only 20 because below that it contains less data to predict anything so I should choose only the top 20 of the number of data

In [ ]:
regionlist = list(df['regio2'].value_counts().head(25).index)
regionlist # top 20 city that contains the most data counts

In [ ]:
df = df[pd.DataFrame(df['regio2'].tolist()).isin(regionlist).any(1).values]
df['regio2'].value_counts()

## Outliers

What we should focus in the first is the data that we want to predict and what relavance most which is 'KaltMiete' and 'WarmMiete'

In [ ]:
df['baseRent'].describe().round(2)

In [ ]:
df['totalRent'].describe().round(2)

Code below I filter and plot the graph repeatly to see the best result and try to remove the outliers that doesn't make sense to the df

In [ ]:
df = df[(df['baseRent'] > 200) & (df['baseRent'] < 8000)]
df = df[(df['totalRent'] > 200) & (df['totalRent'] < 9000)]
df = df[(df['totalRent'] > df['baseRent'])]
df = df[(df['totalRent'] - df['baseRent']) < 500]


In [ ]:
fig = px.scatter(df, x='totalRent', y='baseRent')
fig.show()

In [ ]:
df['livingSpace'].describe()

In [ ]:
df = df[(df['livingSpace'] > 10) & (df['livingSpace'] < 400)]


In [ ]:
fig = px.scatter(df, x='baseRent', y='livingSpace')
fig.show()

### Feature Engineering
We've already created some columns already such as 'numberOfYears'. However, now I'm trying to create more variables for inspect and building a model from new variables later.

Create a new columns for the price per square meter

In [ ]:
df['Pricepm2'] = df['baseRent'] / df['livingSpace']
df['additioncost'] = df['totalRent'] - df['baseRent']

In [ ]:
fig = px.scatter(df, x='totalRent', y='Pricepm2')
fig.show()

### Service Charge

In [ ]:
df['serviceCharge'].describe()

In [ ]:
df = df[(df['serviceCharge'] < 1000)]


In [ ]:
fig = px.scatter(df, x='totalRent', y='serviceCharge')
fig.show()

## Fill all the missing values

In [ ]:
df = df[(df['floor'] >= -1) & (df['floor'] <= 20)] # Floor should contain only basement - 20th floor
df['heatingType'].fillna(df['heatingType'].mode()[0], inplace=True)
df['typeOfFlat'].fillna(df['typeOfFlat'].mode()[0], inplace=True)

In [ ]:
heatinglist = list(df['heatingType'].value_counts().head(10).index)
df = df[pd.DataFrame(df['heatingType'].tolist()).isin(heatinglist).any(1).values]

In [ ]:
df.head()

In [ ]:
for cols in df.columns:
    if df[cols].dtype == 'int64' or df[cols].dtype == 'float64':
        upper_range = df[cols].mean() + 3 * df[cols].std()
        lower_range = df[cols].mean() - 3 * df[cols].std()
        
        indexs = df[(df[cols] > upper_range) | (df[cols] < lower_range)].index
        df = df.drop(indexs)

Drop the 'baseRent' columns because it has a lot of correlation to 'totalRent'

In [ ]:
df.drop(['baseRent'],axis=1,inplace=True)

Checking for the last time, we don't have any missing data left

In [ ]:
missing_values(df,5)

Now we're finished with Data Cleaning Job so we could virtualization to have a better understanding of our dataset.

# Data Virtualization

In [ ]:
df.head()

I love to use correlation map to inspect the dataset. Which  variables has the more correlation to the variables that we want to predict.

In [ ]:
f, ax = plt.subplots(figsize=(12, 12))

sns.heatmap(df.corr().sort_values(by='totalRent',ascending=False), square = True,fmt='.2f' ,annot = True)

From the dataset, variables that irrelevant to 'totalRent' are 'cellar','floor' and 'garden' so I would consider drop it.

In [ ]:
df.drop(['cellar','floor','garden'],axis=1,inplace=True)

### Basic inspection to check the kurtosis and skewness of the data

In [ ]:
fig,ax = plt.subplots(figsize=(10,6))
sns.distplot(df['totalRent'],fit=norm)

In [ ]:
fig,ax = plt.subplots(figsize=(10,6))
sns.distplot(df['livingSpace'],fit=norm)

In [ ]:
df.head()

### Start with the ratio of each city

In [ ]:
countpie = df['regio2'].value_counts()

fig = {
  "data": [
    {
      "values": countpie.values,
      "labels": countpie.index,
      "domain": {"x": [0, .5]},
      "name": "City",
      "hoverinfo":"label+percent+name",
      "hole": .7,
      "type": "pie"
    },],
  "layout": {
        "title":"Pie chart of all the City ratio in the dataset",
    }
}
iplot(fig)

In [ ]:
countpie = df['regio2'].value_counts().iloc[1:,]

fig = {
  "data": [
    {
      "values": countpie.values,
      "labels": countpie.index,
      "domain": {"x": [0, .5]},
      "name": "City",
      "hoverinfo":"label+percent+name",
      "hole": .7,
      "type": "pie"
    },],
  "layout": {
        "title":"Pie chart of all the City ratio in the dataset exclude 'Other'",
    }
}
iplot(fig)

In [ ]:
plotter = df.groupby('regio2')['totalRent'].agg(['mean'])
plotter.columns = ["mean"]
plotter['regio2'] = plotter.index

data = [
    {
        'x': plotter['regio2'],
        'y': plotter['mean'],
        'mode': 'markers+text',
        'text' : plotter['regio2'],
        'textposition' : 'bottom center',
        'marker': {  
            'size': 20,
        }
    }
]

layout = go.Layout(title="Average rental per month", 
                   xaxis=dict(title='City'),
                   yaxis=dict(title='Cost of rental')
                  )
fig = go.Figure(data = data, layout = layout)
iplot(fig, filename='scatter0')

München, Frankfurt am Main, Hamburg, Düsseldorf, Berlin and Köln seem to be the highest rental city

In [ ]:
rentmean = df.groupby(['regio2'])['totalRent'].mean().sort_index()

fig = px.histogram(x = df['regio2'].value_counts().sort_index().index,
                   y = rentmean,
                   color= df['regio2'].value_counts().sort_index().index
             )
fig.update_xaxes(title="City")
fig.update_yaxes(title = "Average rental per month")
fig.show()

In [ ]:
plotter = df.groupby('regio2')['Pricepm2'].agg(['mean'])
plotter.columns = ["mean"]
plotter['regio2'] = plotter.index

data = [
    {
        'x': plotter['regio2'],
        'y': plotter['mean'],
        'mode': 'markers+text',
        'text' : plotter['regio2'],
        'textposition' : 'bottom center',
        'marker': {  
            'size': 20,
        }
    }
]

layout = go.Layout(title="Average rental per month compare by area per square meter", 
                   xaxis=dict(title='City'),
                   yaxis=dict(title='Euro/square meter')
                  )
fig = go.Figure(data = data, layout = layout)
iplot(fig, filename='scatter0')

In [ ]:
plotter = df.groupby('condition')['totalRent'].agg(['mean'])
plotter.columns = ["mean"]
plotter['condition'] = plotter.index

data = [
    {
        'x': plotter['condition'],
        'y': plotter['mean'],
        'mode': 'markers+text',
        'text' : plotter['condition'],
        'textposition' : 'bottom center',
        'marker': {  
            'size': 20,
        }
    }
]

layout = go.Layout(title="Average rental per month group by apartment condition", 
                   xaxis=dict(title='Apartment Condition'),
                   yaxis=dict(title='Cost of rental')
                  )
fig = go.Figure(data = data, layout = layout)
iplot(fig, filename='scatter0')

In [ ]:
plotter = df.groupby('regio2')['livingSpace'].agg(['mean'])
plotter.columns = ["mean"]
plotter['regio2'] = plotter.index

data = [
    {
        'x': plotter['regio2'],
        'y': plotter['mean'],
        'mode': 'markers+text',
        'text' : plotter['regio2'],
        'textposition' : 'bottom center',
        'marker': {  
            'size': 20,
        }
    }
]

layout = go.Layout(title="Average living space group by city", 
                   xaxis=dict(title='City'),
                   yaxis=dict(title='Average Living Space')
                  )
fig = go.Figure(data = data, layout = layout)
iplot(fig, filename='scatter0')

In [ ]:
countpie = df['heatingType'].value_counts()

fig = {
  "data": [
    {
      "values": countpie.values,
      "labels": countpie.index,
      "domain": {"x": [0, .5]},
      "name": "City",
      "hoverinfo":"label+percent+name",
      "hole": .7,
      "type": "pie"
    },],
  "layout": {
        "title":"Pie chart of all the City ratio in the dataset",
    }
}
iplot(fig)

In [ ]:
plotter = df.groupby('heatingType')['totalRent'].agg(['mean'])
plotter.columns = ["mean"]
plotter['heatingType'] = plotter.index

data = [
    {
        'x': plotter['heatingType'],
        'y': plotter['mean'],
        'mode': 'markers+text',
        'text' : plotter['heatingType'],
        'textposition' : 'bottom center',
        'marker': {  
            'size': 20,
        }
    }
]

layout = go.Layout(title="Average rental group by heating type", 
                   xaxis=dict(title='Heating Type'),
                   yaxis=dict(title='Average Rental Cost')
                  )
fig = go.Figure(data = data, layout = layout)
iplot(fig, filename='scatter0')

In [ ]:
countpie = df['newlyConst'].value_counts()
countpie = countpie.sort_index() 
fig = {
  "data": [
    {
      "values": countpie.values,
      "labels": ['False','True'],
      "domain": {"x": [0, .5]},
      "hoverinfo":"label+percent+name",
      "hole": .3,
      "type": "pie"
    },],
  "layout": {
        "title":"Percentage of the residence is newly constructed or not",
    }
}
iplot(fig)

Most of the apartment are newly constructed. Then, I want to know the difference of a price between refurbrished are having a big gap or not

In [ ]:
constructmean = df.groupby(['newlyConst'])['totalRent'].mean().sort_index()

fig = px.histogram(x = df['newlyConst'].value_counts().sort_index().index,
                   y = constructmean,
                   color= df['newlyConst'].value_counts().sort_index().index
             )
fig.update_xaxes(title="Newly construct or not")
fig.update_yaxes(title = "Rental Cost")
fig.show()

So the difference between refurbrished or newly construct have more rental earning per month at 600 Euro.

In [ ]:
plotter = df.groupby('regio2')['additioncost'].agg(['mean'])
plotter.columns = ["mean"]
plotter['regio2'] = plotter.index

data = [
    {
        'x': plotter['regio2'],
        'y': plotter['mean'],
        'mode': 'markers+text',
        'text' : plotter['regio2'],
        'textposition' : 'bottom center',
        'marker': {  
            'size': 20,
        }
    }
]

layout = go.Layout(title="Average Addition Cost Per Month (Warmmiete - Kaltmiete)", 
                   xaxis=dict(title='City'),
                   yaxis=dict(title='Additional Cost per month')
                  )
fig = go.Figure(data = data, layout = layout)
iplot(fig, filename='scatter0')

In [ ]:
plotter = df.groupby('typeOfFlat')['totalRent'].agg(['mean'])
plotter.columns = ["mean"]
plotter['typeOfFlat'] = plotter.index

data = [
    {
        'x': plotter['typeOfFlat'],
        'y': plotter['mean'],
        'mode': 'markers+text',
        'text' : plotter['typeOfFlat'],
        'textposition' : 'bottom center',
        'marker': {  
            'size': 20,
        }
    }
]

layout = go.Layout(title="Type of Apartment and Average Rental Cost Per Month", 
                   xaxis=dict(title='Type Of Rental'),
                   yaxis=dict(title='Average rental type cost per month')
                  )
fig = go.Figure(data = data, layout = layout)
iplot(fig, filename='scatter0')

We might want to seperate the type of rental kind because some of it cost very high per month.

In [ ]:
countpie = df['hasKitchen'].value_counts()

fig = {
  "data": [
    {
      "values": countpie.values,
      "labels": countpie.index,
      "domain": {"x": [0, .5]},
      "name": "City",
      "hoverinfo":"label+percent+name",
      "hole": .7,
      "type": "pie"
    },],
  "layout": {
        "title":"Pie chart of the Apartment has kitchen or not",
    }
}
iplot(fig)

Most of the place is not inclde kitchen

In [ ]:
constructmean = df.groupby(['hasKitchen'])['totalRent'].mean().sort_index()

fig = px.histogram(x = df['hasKitchen'].value_counts().sort_index().index,
                   y = constructmean,
                   color= df['hasKitchen'].value_counts().sort_index().index
             )
             
fig.update_xaxes(title="Has Kitchen")
fig.update_yaxes(title = "Cost of Rental")
fig.show()

Further opinion, we could create more meaningful virtualization such as seperate rental types or others to make it clearer for the trend of rental cost in Germany

# Preparing the Data for Machine Learning

Copying the data

In [ ]:
predict_df = df.copy()
predict_df.head()

In [ ]:
predict_df.drop(columns=['yearConstructed'],inplace=True)

I want to normalization the columns that's int or float type

In [ ]:
for cols in predict_df.columns:
    if predict_df[cols].dtype == 'int64' or predict_df[cols].dtype == 'float64':
        if cols != 'totalRent':
            predict_df[cols] = ((predict_df[cols] - predict_df[cols].mean())/(predict_df[cols].std()))

predict_df            

If it's an object or bool type (True,False). Code below will create the dummies for all of the categorical.

In [ ]:
columns = []
for cols in predict_df.columns:
    if predict_df[cols].dtype == 'object' or predict_df[cols].dtype == 'bool':
        columns.append(cols)
        
dummies_feature = pd.get_dummies(predict_df[columns])
dummies_feature.head()

Combine those columns together and drop the categorical columns that we created the dummies

In [ ]:
predict_df = pd.concat([predict_df, dummies_feature], axis=1)
predict_df = predict_df.drop(columns=columns)
predict_df.head()

Move the 'totalRent' column to the first column to make it easier for split the data.

In [ ]:
move = predict_df['totalRent']
predict_df.drop(labels=['totalRent'], axis=1,inplace = True)
predict_df.insert(0, 'totalRent', move)
predict_df.head()

### Spliting the data into train and test

In [ ]:
X = predict_df.iloc[:,1:].values # Select all the columns, that's not totalRent
y = predict_df.iloc[:,0].values # Select only totalRent

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 123)

## Light Gradient Boost
I've found this model is like Xgboost and it's run much faster, so now I'm trying as much as I could to use this libraries to improve my skills

In [ ]:
d_train = lgb.Dataset(x_train, label=y_train) # Load the dataset and test

# parameters for this model
params = {
        'n_estimators': 10000,
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'max_depth': -1,
        'learning_rate': 0.01,
        'subsample': 0.72,
        'subsample_freq': 4,
        'feature_fraction': 0.4,
        'lambda_l1': 1,
        'lambda_l2': 1,
        'seed': 46,
        }

clf = lgb.train(params, d_train, 100)

Check the data is what'we expected or not.

In [ ]:
y_pred = clf.predict(x_test)
y_pred

Create the RMSE function to check the score.

In [ ]:
def compute_rmse(model, X, y_true, name):
    y_pred = model.predict(X)
    mse = mean_squared_error(y_true, y_pred)
    rmse = pow(mse,0.5)
    print(f'Root Mean Squared Error for {name}: {rmse}')

In [ ]:
compute_rmse(clf, x_train, y_train, 'Training Set')
compute_rmse(clf, x_test, y_test, 'Test Set')

The result of the LGBM is working pretty well and we could use for this model in real life situation and help the other to know how much money their should pay for the apartment they're looking for.

# Summary

This is the end of the kernel, if you love this kernel or could study something from this please upvote! it means a lot for my future opportunity. Moreover, feel free to comment on my mistakes because it would be surely help me to improve my mistakes.

Thanks for viewing!